In [18]:
# Importing the necessary Python libraries
import numpy as np
import pandas as pd
from statistics import mean
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, mean_absolute_error, mean_squared_error, r2_score

# Importing the regression algorithms
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor

In [52]:
df_original = pd.read_csv('movies.csv')
df_original['profit'] = df_original['gross']-df_original['budget']
df_original.head()

,name,rating,genre,year,released,score,votes,director,writer,star,country,budget,gross,company,runtime,profit
0,The Shining,R,Drama,1980,"June 13, 1980 (United States)",8.4,927000.0,Stanley Kubrick,Stephen King,Jack Nicholson,United Kingdom,19000000.0,46998772.0,Warner Bros.,146.0,27998772.0
1,The Blue Lagoon,R,Adventure,1980,"July 2, 1980 (United States)",5.8,65000.0,Randal Kleiser,Henry De Vere Stacpoole,Brooke Shields,United States,4500000.0,58853106.0,Columbia Pictures,104.0,54353106.0
2,Star Wars: Episode V - The Empire Strikes Back,PG,Action,1980,"June 20, 1980 (United States)",8.7,1200000.0,Irvin Kershner,Leigh Brackett,Mark Hamill,United States,18000000.0,538375067.0,Lucasfilm,124.0,520375067.0
3,Airplane!,PG,Comedy,1980,"July 2, 1980 (United States)",7.7,221000.0,Jim Abrahams,Jim Abrahams,Robert Hays,United States,3500000.0,83453539.0,Paramount Pictures,88.0,79953539.0
4,Caddyshack,R,Comedy,1980,"July 25, 1980 (United States)",7.3,108000.0,Harold Ramis,Brian Doyle-Murray,Chevy Chase,United States,6000000.0,39846344.0,Orion Pictures,98.0,33846344.0


In [53]:
# len(df_original['genre'].unique())
df = df_original[df_original['year'] > 1999]
df
# len(df['company'].unique())

,name,rating,genre,year,released,score,votes,director,writer,star,country,budget,gross,company,runtime,profit
3643,Almost Famous,R,Adventure,2000,"September 22, 2000 (United States)",7.9,260000.0,Cameron Crowe,Cameron Crowe,Billy Crudup,United States,60000000.0,47386287.0,Columbia Pictures,122.0,-12613713.0
3644,American Psycho,R,Comedy,2000,"April 14, 2000 (United States)",7.6,514000.0,Mary Harron,Bret Easton Ellis,Christian Bale,United States,7000000.0,34266564.0,Am Psycho Productions,101.0,27266564.0
3645,Gladiator,R,Action,2000,"May 5, 2000 (United States)",8.5,1400000.0,Ridley Scott,David Franzoni,Russell Crowe,United States,103000000.0,465380802.0,Dreamworks Pictures,155.0,362380802.0
3646,Snatch,R,Comedy,2000,"January 19, 2001 (United States)",8.3,800000.0,Guy Ritchie,Guy Ritchie,Jason Statham,United Kingdom,NaN,83557872.0,Columbia Pictures,102.0,NaN
3647,Requiem for a Dream,Unrated,Drama,2000,"December 15, 2000 (United States)",8.3,786000.0,Darren Aronofsky,Hubert Selby Jr.,Ellen Burstyn,United States,4500000.0,7390108.0,Artisan Entertainment,102.0,2890108.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7663,More to Life,NaN,Drama,2020,"October 23, 2020 (United States)",3.1,18.0,Joseph Ebanks,Joseph Ebanks,Shannon Bond,United States,7000.0,NaN,NaN,90.0,NaN
7664,Dream Round,NaN,Comedy,2020,"February 7, 2020 (United States)",4.7,36.0,Dusty Dukatz,Lisa Huston,Michael Saquella,United States,NaN,NaN,Cactus Blue Entertainment,90.0,NaN
7665,Saving Mbango,NaN,Drama,2020,"April 27, 2020 (Cameroon)",5.7,29.0,Nkanya Nkwai,Lynno Lovert,Onyama Laura,United States,58750.0,NaN,Embi Productions,NaN,NaN
7666,It's Just Us,NaN,Drama,2020,"October 1, 2020 (United States)",NaN,NaN,James Randall,James Randall,Christina Roz,United States,15000.0,NaN,NaN,120.0,NaN


In [21]:
len(df['country'].unique())

52

In [55]:
df = df[['votes','profit','budget','genre', 'rating', 'score', 'runtime', 'year']]

df = df.dropna()
df

,votes,profit,budget,genre,rating,score,runtime,year
3643,260000.0,-12613713.0,60000000.0,Adventure,R,7.9,122.0,2000
3644,514000.0,27266564.0,7000000.0,Comedy,R,7.6,101.0,2000
3645,1400000.0,362380802.0,103000000.0,Action,R,8.5,155.0,2000
3647,786000.0,2890108.0,4500000.0,Drama,Unrated,8.3,102.0,2000
3648,1200000.0,31047078.0,9000000.0,Mystery,R,8.4,113.0,2000
...,...,...,...,...,...,...,...,...
7648,140000.0,336505244.0,90000000.0,Action,R,6.6,124.0,2020
7649,102000.0,234715683.0,85000000.0,Action,PG,6.5,99.0,2020
7650,53000.0,70487753.0,175000000.0,Adventure,PG,5.6,101.0,2020
7651,42000.0,-23894503.0,135000000.0,Adventure,PG,6.8,100.0,2020


In [56]:
from sklearn.preprocessing import OneHotEncoder
from category_encoders import OneHotEncoder

# Defining the OneHotEncoders for the genre columns
genre_encoder = OneHotEncoder(use_cat_names = True, handle_unknown = 'ignore')

# Getting the one-hot encoded dummies for genre
genre_dummies = genre_encoder.fit_transform(df['genre'])

# Concatenating the genre dummies to the original dataframe
df = pd.concat([df, genre_dummies], axis = 1)

# Dropping the original genre columns
df.drop(columns = ['genre'], inplace = True)

In [57]:
# Defining the OneHotEncoders for the rating columns
rating_encoder = OneHotEncoder(use_cat_names = True, handle_unknown = 'ignore')

# Getting the one-hot encoded dummies for rating
rating_dummies = rating_encoder.fit_transform(df['rating'])

# Concatenating the rating dummies to the original dataframe
df = pd.concat([df, rating_dummies], axis = 1)

# Dropping the original rating columns
df.drop(columns = ['rating'], inplace = True)

In [58]:
# Importing warnings to suppress warnings
import warnings
warnings.filterwarnings('ignore')

# Loading in the cleaned dataset
df_clean = df

## Regression Models

In [59]:
# Splitting the predictor value from the remainder of the dataset
X = df_clean.drop(columns = ['profit'])
y = df_clean[['profit']]

In [63]:
# Creating a reusable function for churning through all five regression algorithms
def generate_regression_model(X, y, model_algorithm, hyperparameters, needs_scaled = False):
    """
    Generating everything required for training and validation of a regression model

    Args:
        - X (Pandas DataFrame): A DataFrame containing the cleaned training data
        - y (Pandas DataFrame): A DataFrame containing the target values correlated to the X training data
        - model_algorithm (object): A model algorithm that will be trained against the X and y data
        - hyperparameters (dict): A dictionary containing all the hyperparameters to test the model with
        - needs_scaled (Boolean): A boolean value that indicates whether or not the input dataset
    """
    
    # Performing a scaling on the data if required
    if needs_scaled:
        
        # Instantiating the StandardScaler
        scaler = StandardScaler()
        
        # Performing a fit_transform on the dataset
        scaled_features = scaler.fit_transform(X)
        
        # Transforming the StandardScaler output back into a Pandas DataFrame
        X = pd.DataFrame(scaled_features, index = X.index, columns = X.columns)
        
    # Instantiating a GridSearch object with the inputted model algorithm and hyperparameters
    gridsearchcv = GridSearchCV(estimator = model_algorithm,
                                param_grid = hyperparameters)
    
    # Fitting the training data to the GridSearch object
    gridsearchcv.fit(X, y)
    
    # Printing out the best hyperparameters
    print(f'Best hyperparameters: {gridsearchcv.best_params_}')
    
    # Instantiating a new model object with the ideal hyperparameters from the GridSearch job
    model_algorithm.set_params(**gridsearchcv.best_params_)
    
    # Creating a container to hold each set of validation metrics
    mae_scores, rmse_scores, r2_scores = [], [], []
    
    # Instantiating the K-Fold cross validation object
    k_fold = KFold(n_splits = 5)
    
    # Iterating through each of the folds in K-Fold
    for train_index, val_index in k_fold.split(X):

        # Splitting the training set from the validation set for this specific fold
        X_train, X_val = X.iloc[train_index, :], X.iloc[val_index, :]
        y_train, y_val = y.iloc[train_index], y.iloc[val_index]
        
        # Fitting the X_train and y_train datasets to the model algorithm
        model_algorithm.fit(X_train, y_train)

        # Getting inferential predictions for the validation dataset
        val_preds = model_algorithm.predict(X_val)

        # Generating validation metrics by comparing the inferential predictions (val_preds) to the actuals (y_val)
        val_mae_score = mean_absolute_error(y_val, val_preds)
        val_mse_score = mean_squared_error(y_val, val_preds)
        val_rmse_score = mean_squared_error(y_val, val_preds, squared = False)
        val_r2_score = r2_score(y_val, val_preds)
        
        # Appending the validation scores to the respective validation metric container
        mae_scores.append(val_mae_score)
        rmse_scores.append(val_rmse_score)
        r2_scores.append(val_r2_score)
        
    # Getting the average (mean) of each validation score
    average_mae = mean(mae_scores)
    average_rmse = mean(rmse_scores)
    average_r2 = mean(r2_scores)
     
    # Printing out the average validation metrics
    print(f'Average mean absolute error: {average_mae}')
    print(f'Average root mean squared error: {average_rmse}')
    print(f'Average R2 score: {average_r2}')

## 1. Linear Regression

In [65]:
# Setting the hyperparameter grid for the Linear Regression algorithm
linear_reg_params = {
    # No hyperparameters to tune!
}

# Instantiating the Linear Regression algorithm object
linear_reg_algorithm = LinearRegression()

# Feeding the algorithm into the reusable regression function
generate_regression_model(X = X,
                          y = y,
                          model_algorithm = linear_reg_algorithm,
                          hyperparameters = linear_reg_params)



Best hyperparameters: {}
Average mean absolute error: 72737627.82466137
Average root mean squared error: 125390091.66339943
Average R2 score: 0.4958562376385568


In [66]:
X.columns#[-9]

Index(['votes', 'budget', 'score', 'runtime', 'year', 'genre_Adventure',
       'genre_Comedy', 'genre_Action', 'genre_Drama', 'genre_Mystery',
       'genre_Biography', 'genre_Animation', 'genre_Crime', 'genre_Horror',
       'genre_Romance', 'genre_Fantasy', 'genre_Family', 'genre_Thriller',
       'genre_Sci-Fi', 'rating_R', 'rating_Unrated', 'rating_PG-13',
       'rating_PG', 'rating_G', 'rating_Not Rated', 'rating_NC-17',
       'rating_TV-MA'],
      dtype='object')

In [44]:
list = (linear_reg_algorithm.coef_).tolist()
np.sort(list)
# len(list)

array([[-1.14393495e+08, -4.39272795e+07, -2.61159971e+07,
        -2.23899817e+07, -1.70208526e+07, -9.15766842e+06,
        -6.44988868e+06, -5.08775151e+06, -4.85703959e+06,
         0.00000000e+00,  1.21142112e+00,  4.27100212e+02,
         8.26247498e+04,  1.13449079e+05,  1.09867431e+06,
         1.73561172e+06,  2.30932933e+06,  4.50605375e+06,
         1.18843676e+07,  1.58014221e+07,  1.58902147e+07,
         1.73869826e+07,  1.79859030e+07,  2.30205493e+07,
         3.58468550e+07,  4.33323271e+07,  5.59405044e+07]])

In [43]:
(linear_reg_algorithm.coef_)

array([[ 4.27100212e+02,  1.21142112e+00, -4.85703959e+06,
         8.26247498e+04,  2.30932933e+06,  1.73869826e+07,
         1.58902147e+07, -9.15766842e+06,  1.13449079e+05,
        -4.39272795e+07, -6.44988868e+06,  5.59405044e+07,
         4.50605375e+06,  3.58468550e+07, -1.70208526e+07,
         2.30205493e+07, -5.08775151e+06,  4.33323271e+07,
        -1.14393495e+08, -2.23899817e+07, -2.61159971e+07,
         1.09867431e+06,  1.79859030e+07,  1.58014221e+07,
         1.73561172e+06,  1.18843676e+07,  0.00000000e+00]])

## 2. Lasso Regression

In [66]:
# Setting the hyperparameter grid for the Lasso Regression algorithm
lasso_reg_params = {
    'alpha': np.linspace(0.2, 2, 25)
}
# Instantiating the Lasso Regression algorithm object
lasso_reg_algorithm = Lasso()
# Feeding the algorithm into the reusable regression function
generate_regression_model(X = X,
                          y = y,
                          model_algorithm = lasso_reg_algorithm,
                          hyperparameters = lasso_reg_params,
                          needs_scaled = True)

Best hyperparameters: {'alpha': 2.0}
Average mean absolute error: 72719255.97205494
Average root mean squared error: 125359067.14649057
Average R2 score: 0.49608355223719075


## 3. CatBoost Regressor

In [67]:
# Setting the hyperparameter grid for the CatBoost Regressor algorithm
catboost_reg_params = {
    'depth': [1, 2, 3,4,5,6],
    'learning_rate': [0.001, 0.002, 0.003],
    'iterations': [1, 2, 5,6,7,8,9,10]
}
# Instantiating the CatBoost Classifier algorithm object
catboost_reg_algorithm = CatBoostRegressor(silent = True)
# Feeding the algorithm into the reusable regression function
generate_regression_model(X = X,
                          y = y,
                          model_algorithm = catboost_reg_algorithm,
                          hyperparameters = catboost_reg_params)

Best hyperparameters: {'depth': 6, 'iterations': 10, 'learning_rate': 0.003}
Average mean absolute error: 108503583.73879735
Average root mean squared error: 180156983.7123295
Average R2 score: -0.019585281464866332


In [3]:
df_original['year']

0       1980
1       1980
2       1980
3       1980
4       1980
        ... 
7663    2020
7664    2020
7665    2020
7666    2020
7667    2020
Name: year, Length: 7668, dtype: int64